# Week 3 - Evaluating Toronto Neighborhoods
## Gary Netherton - July 4, 2020

### Initial download of Toronto data from Wikipedia
### Use Pandas to scrape the web page and create a panda dataframe

In [2]:
# import required libraries
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np


In [3]:
# Use Pandas to scrape HTML table from Wikipedia; create a dataframe from results
data = pd.read_html('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') # read Wikipedia page provided by Coursera
df = data[0]
print(df.shape)
df.head()

(180, 3)


Postal Code           Borough               Neighborhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

In [4]:
# Remove rows where the Borough has not been assigned
df=df[df['Borough']!='Not assigned']
df.shape

(103, 3)

### For next portion, collect the latitude and longitude of each Postal Code
### Combine this data with the data from the initial data frame to form one dataframe with all information

In [5]:
# Need to find latitude and longitude of each neighborhood
# Per the instructions, the geocoder package is unreliable.  I was unable to make it work in two different environments.
# Using the provided CSV file to create a dataframe of postal codes with latitude and longitude
geodata = pd.read_csv(r'C:\Users\garyn\OneDrive\Documents\Coursera_Labs\Week_3_labs\Coursera_Capstone_Week3\Geospatial_Coordinates.csv')
geodata.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [6]:
# Using merge to combine original dataframe with latitude and longitude dataframe
df = pd.merge(df, geodata, on='Postal Code', how='outer')
df.head()

Postal Code           Borough                                 Neighborhood  \
0         M3A        North York                                    Parkwoods   
1         M4A        North York                             Victoria Village   
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
3         M6A        North York             Lawrence Manor, Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494

### Download the appropriate libraries to use the Foursquare API to explore
### Use Venues similar to the New York lab earlier in the course

In [26]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [8]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.657952, -79.387383], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [9]:
# Setup the Foursquare API information

CLIENT_ID = 'BRBDGCKR00L1JCYN5RD3ZDJ2BNOJ5YOVAN3F2KZNVVUVB3VW' # your Foursquare ID
CLIENT_SECRET = '5N1FQNDEEVF0LSK4A1SFM41P4KFSGECE1T23QNK20K1SD4PR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BRBDGCKR00L1JCYN5RD3ZDJ2BNOJ5YOVAN3F2KZNVVUVB3VW
CLIENT_SECRET:5N1FQNDEEVF0LSK4A1SFM41P4KFSGECE1T23QNK20K1SD4PR


In [10]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [11]:
# Create function to collect venue information from all neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
# Collect the information from Foursquare for Toronta neighborhoods
toronto_venues = getNearbyVenues(df['Neighborhood'],
                                   df['Latitude'],
                                   df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

### Explore the venue information that downloaded from Foursquare

In [13]:
# Get shape and head() of dataframe
print(toronto_venues.shape)
toronto_venues.head()

(2129, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0         Parkwoods              43.753259              -79.329656   
1         Parkwoods              43.753259              -79.329656   
2  Victoria Village              43.725882              -79.315572   
3  Victoria Village              43.725882              -79.315572   
4  Victoria Village              43.725882              -79.315572   

                    Venue  Venue Latitude  Venue Longitude  \
0         Brookbanks Park       43.751976       -79.332140   
1           Variety Store       43.751974       -79.333114   
2  Victoria Village Arena       43.723481       -79.315635   
3               Portugril       43.725819       -79.312785   
4             Tim Hortons       43.725517       -79.313103   

          Venue Category  
0                   Park  
1      Food & Drink Shop  
2           Hockey Arena  
3  Portuguese Restaurant  
4            Coffee Shop

In [14]:
# Number of venues returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Agincourt                                                               5   
Alderwood, Long Branch                                                  8   
Bathurst Manor, Wilson Heights, Downsview North                        20   
Bayview Village                                                         4   
Bedford Park, Lawrence Manor East                                      24   
Berczy Park                                                            58   
Birch Cliff, Cliffside West                                             4   
Brockton, Parkdale Village, Exhibition Place                           22   
Business reply mail Processing Centre, South Ce...                     19   
CN Tower, King and Spadina, Railway Lands, Harb...                     15   
Caledonia-Fairbanks                                                     4   
Canada Post Gateway Processing Centre                                  13   
Cedarbrae                                                               9   
Central Bay Street                                                     65   
Christie                                                               17   
Church and Wellesley                                                   75   
Clarks Corners, Tam O'Shanter, Sullivan                                12   
Cliffside, Cliffcrest, Scarborough Village West                         3   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             35   
Davisville North                                                        8   
Del Ray, Mount Dennis, Keelsdale and Silverthorn                        5   
Don Mills                                                              25   
Dorset Park, Wexford Heights, Scarborough Town ...                      7   
Downsview                                                              15   
Dufferin, Dovercourt Village                                           15   
East Toronto, Broadview North (Old East York)                           3   
Eringate, Bloordale Gardens, Old Burnhamthorpe,...                      8   
Fairview, Henry Farm, Oriole                                           65   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         4   
Garden District, Ryerson                                              100   
Glencairn                                                               5   
Golden Mile, Clairlea, Oakridge                                        10   
Guildwood, Morningside, West Hill                                       8   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          25   
Hillcrest Village                                                       4   
Humber Summit                                                           2   
Humberlea, Emery                                                        1   
Humewood-Cedarvale                                                      4   
India Bazaar, The Beaches West                                         23   
Kennedy Park, Ionview, East Birchmount Park                             4   
Kensington Market, Chinatown, Grange Park                              60   
Kingsview Village, St. Phillips, Martin Grove G...                      2   
Lawrence Manor, Lawrence Heights                                       14   
Lawrence Park                                                           3   
Leaside                                                                33   
Little Portugal, Trinity                                               44   
Malvern, Rouge                                                          1   
Milliken, Agincourt North, Steeles East, L'Amor.

### Summarize unique categories from returned venues

In [15]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 268 uniques categories.


### Analyze Each Neighborhood in Toronto

In [16]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
print(toronto_onehot.shape)
toronto_onehot.head()

(2129, 268)


Yoga Studio  Accessories Store  Afghan Restaurant  Airport  \
0            0                  0                  0        0   
1            0                  0                  0        0   
2            0                  0                  0        0   
3            0                  0                  0        0   
4            0                  0                  0        0   

   Airport Food Court  Airport Lounge  Airport Service  Airport Terminal  \
0                   0               0                0                 0   
1                   0               0                0                 0   
2                   0               0                0                 0   
3                   0               0                0                 0   
4                   0               0                0                 0   

   American Restaurant  Antique Shop  Aquarium  Art Gallery  Art Museum  \
0                    0             0         0            0           0   
1                    0             0         0            0           0   
2                    0             0         0            0           0   
3                    0             0         0            0           0   
4                    0             0         0            0           0   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Auto Garage  \
0                    0                 0                   0            0   
1                    0                 0                   0            0   
2                    0                 0                   0            0   
3                    0                 0                   0            0   
4                    0                 0                   0            0   

   Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  \
0              0          0           0           0       0     0    0   
1              0          0           0           0       0     0    0   
2              0          0           0           0       0     0    0   
3              0          0           0           0       0     0    0   
4              0          0           0           0       0     0    0   

   Baseball Field  Baseball Stadium  Basketball Stadium  Beach  \
0               0                 0                   0      0   
1               0                 0                   0      0   
2               0                 0                   0      0   
3               0                 0                   0      0   
4               0                 0                   0      0   

   Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  Bike Shop  \
0                0         0           0                   0          0   
1                0         0           0                   0          0   
2                0         0           0                   0          0   
3                0         0           0                   0          0   
4                0         0           0                   0          0   

   Bistro  Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  \
0       0              0          0         0                     0   
1       0              0          0         0                     0   
2       0              0          0         0                     0   
3       0              0          0         0                     0   
4       0              0          0         0                     0   

   Breakfast Spot  Brewery  Bridal Shop  Bubble Tea Shop  Building  \
0               0        0            0                0         0   
1               0        0            0                0         0   
2               0        0            0                0         0   
3               0        0            0                0         0   
4               0        0            0                0         0   

   Burger Joint  Burrito Place  Bus Line  Bus Station  Business Service  \
0             0          

### Similar to the New Yort lab, group rows by neighborhood by taking the mean
### of the frequency of occurrence of each category

In [17]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()

(93, 268)


Neighborhood  Yoga Studio  \
0                                        Agincourt          0.0   
1                           Alderwood, Long Branch          0.0   
2  Bathurst Manor, Wilson Heights, Downsview North          0.0   
3                                  Bayview Village          0.0   
4                Bedford Park, Lawrence Manor East          0.0   

   Accessories Store  Afghan Restaurant  Airport  Airport Food Court  \
0                0.0                0.0      0.0                 0.0   
1                0.0                0.0      0.0                 0.0   
2                0.0                0.0      0.0                 0.0   
3                0.0                0.0      0.0                 0.0   
4                0.0                0.0      0.0                 0.0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0             0.0              0.0               0.0             0.000000   
1             0.0              0.0               0.0             0.000000   
2             0.0              0.0               0.0             0.000000   
3             0.0              0.0               0.0             0.000000   
4             0.0              0.0               0.0             0.041667   

   Antique Shop  Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  \
0           0.0       0.0          0.0         0.0                  0.0   
1           0.0       0.0          0.0         0.0                  0.0   
2           0.0       0.0          0.0         0.0                  0.0   
3           0.0       0.0          0.0         0.0                  0.0   
4           0.0       0.0          0.0         0.0                  0.0   

   Asian Restaurant  Athletics & Sports  Auto Garage  Auto Workshop  \
0               0.0                 0.0          0.0            0.0   
1               0.0                 0.0          0.0            0.0   
2               0.0                 0.0          0.0            0.0   
3               0.0                 0.0          0.0            0.0   
4               0.0                 0.0          0.0            0.0   

   BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  Baseball Field  \
0        0.0         0.0         0.0     0.0  0.00  0.0             0.0   
1        0.0         0.0         0.0     0.0  0.00  0.0             0.0   
2        0.0         0.0         0.0     0.0  0.10  0.0             0.0   
3        0.0         0.0         0.0     0.0  0.25  0.0             0.0   
4        0.0         0.0         0.0     0.0  0.00  0.0             0.0   

   Baseball Stadium  Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  \
0               0.0                 0.0    0.0              0.0       0.0   
1               0.0                 0.0    0.0              0.0       0.0   
2               0.0                 0.0    0.0              0.0       0.0   
3               0.0                 0.0    0.0              0.0       0.0   
4               0.0                 0.0    0.0              0.0       0.0   

   Beer Store  Belgian Restaurant  Bike Shop  Bistro  Boat or Ferry  \
0         0.0                 0.0        0.0     0.0            0.0   
1         0.0                 0.0        0.0     0.0            0.0   
2         0.0                 0.0        0.0     0.0            0.0   
3         0.0                 0.0        0.0     0.0            0.0   
4         0.0                 0.0        0.0     0.0            0.0   

   Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  Brewery  \
0        0.0       0.0                   0.0             0.2      0.0   
1        0.0       0.0                   0.0             0.0      0.0   
2        0.0       0.0                   0.0             0.0      0.0   
3        0.0       0.0                   0.0             0.0      0.0   
4        0.0       0.0                   0.0             0.0      0.0   

   Bridal Shop  Bubble Tea Shop  Building  Burger Joint  Burrito Place  \
0      

### Print each neighborhood with top 5 most common venues
### Then, add that to a pandas dataframe

In [18]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Clothing Store   0.2
1             Breakfast Spot   0.2
2                     Lounge   0.2
3  Latin American Restaurant   0.2
4               Skating Rink   0.2


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.25
1  Sandwich Place  0.12
2             Gym  0.12
3            Pool  0.12
4    Dance Studio  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                       Bank  0.10
1                Coffee Shop  0.10
2  Middle Eastern Restaurant  0.05
3           Sushi Restaurant  0.05
4                Supermarket  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2   Chinese Restaurant  0.25
3                 Bank  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0               Restaurant  0.08
1       Italian R

                        venue  freq
0                   Drugstore   0.5
1         Rental Car Location   0.5
2                 Yoga Studio   0.0
3  Modern European Restaurant   0.0
4           Mobile Phone Shop   0.0


----Northwood Park, York University----
                    venue  freq
0  Furniture / Home Store  0.25
1          Massage Studio  0.12
2    Caribbean Restaurant  0.12
3           Metro Station  0.12
4                     Bar  0.12


----Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East----
                        venue  freq
0              Baseball Field   0.5
1                Home Service   0.5
2                 Yoga Studio   0.0
3               Metro Station   0.0
4  Modern European Restaurant   0.0


----Parkdale, Roncesvalles----
            venue  freq
0  Breakfast Spot  0.13
1       Gift Shop  0.13
2     Coffee Shop  0.07
3            Bank  0.07
4   Movie Theater  0.07


----Parkvi

In [19]:
# Insert the most common venues data into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
# The following code creates a new dataframe that will display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Agincourt                Lounge   
1                           Alderwood, Long Branch           Pizza Place   
2  Bathurst Manor, Wilson Heights, Downsview North           Coffee Shop   
3                                  Bayview Village                  Café   
4                Bedford Park, Lawrence Manor East           Coffee Shop   

       2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0  Latin American Restaurant        Breakfast Spot          Skating Rink   
1               Dance Studio                   Pub                   Gym   
2                       Bank    Frozen Yogurt Shop         Shopping Mall   
3                       Bank    Chinese Restaurant   Japanese Restaurant   
4                 Restaurant        Sandwich Place    Italian Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0        Clothing Store             Drugstore        Discount Store   
1           Coffee Shop        Sandwich Place                  Pool   
2           Bridal Shop        Sandwich Place                 Diner   
3         Women's Store        Discount Store   Distribution Center   
4       Thai Restaurant              Pharmacy           Pizza Place   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0   Distribution Center               Dog Run       Doner Restaurant  
1          Dessert Shop    Dim Sum Restaurant                  Diner  
2            Restaurant         Deli / Bodega            Supermarket  
3               Dog Run      Doner Restaurant             Donut Shop  
4                   Pub                  Café                Butcher

### Now to cluster the neighborhoods into 5 clusters
### Picking the number 5 semi-randomly ("semi-randomly" as it was used in the New York lab)

In [21]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [28]:
# add clustering labels for new dataframe to include the cluster and its top 10 venus
neighborhoods_venues_sorted.insert(0, df[list("ABCD")] = df[list("ABCD")].fillna(0.0).astype(int), kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

Postal Code           Borough                                 Neighborhood  \
0         M3A        North York                                    Parkwoods   
1         M4A        North York                             Victoria Village   
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
3         M6A        North York             Lawrence Manor, Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  Cluster Labels  1st Most Common Venue  \
0  43.753259 -79.329656             1.0      Food & Drink Shop   
1  43.725882 -79.315572             2.0  Portuguese Restaurant   
2  43.654260 -79.360636             2.0            Coffee Shop   
3  43.718518 -79.464763             2.0         Clothing Store   
4  43.662301 -79.389494             2.0            Coffee Shop   

  2nd Most Common Venue   3rd Most Common Venue 4th Most Common Venue  \
0                  Park           Women's Store                 Diner   
1           Coffee Shop       French Restaurant          Hockey Arena   
2                  Park                     Pub                Bakery   
3     Accessories Store  Furniture / Home Store              Boutique   
4                 Diner        Sushi Restaurant           Yoga Studio   

   5th Most Common Venue 6th Most Common Venue        7th Most Common Venue  \
0         Discount Store   Distribution Center                      Dog Run   
1           Intersection           Pizza Place  Eastern European Restaurant   
2                Theater        Breakfast Spot                         Café   
3  Vietnamese Restaurant    Miscellaneous Shop                  Coffee Shop   
4     College Auditorium                   Bar                     Beer Bar   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0      Doner Restaurant            Donut Shop              Drugstore  
1             Drugstore            Donut Shop      Electronics Store  
2        Ice Cream Shop                   Spa             Shoe Store  
3           Event Space      Doner Restaurant                  Diner  
4         Smoothie Shop        Sandwich Place          Burrito Place

### Visualize the clusters

In [35]:
# Adminstrative information needed to map

toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].fillna(0.0).astype(int)

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="CA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [36]:
# create map
import folium



map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]



# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Finally, examine the clusters

In [37]:
# CLUSTER #1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
5     Etobicoke               0                   NaN                   NaN   
11    Etobicoke               0                   NaN                   NaN   
32  Scarborough               0            Playground         Women's Store   
45   North York               0                   NaN                   NaN   
52   North York               0                   NaN                   NaN   
64         York               0                   NaN                   NaN   
85  Scarborough               0                  Park            Playground   
95  Scarborough               0                   NaN                   NaN   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
5                    NaN                   NaN                   NaN   
11                   NaN                   NaN                   NaN   
32            Donut Shop    Dim Sum Restaurant                 Diner   
45                   NaN                   NaN                   NaN   
52                   NaN                   NaN                   NaN   
64                   NaN                   NaN                   NaN   
85      Sculpture Garden               Dog Run          Dessert Shop   
95                   NaN                   NaN                   NaN   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
5                    NaN                   NaN                   NaN   
11                   NaN                   NaN                   NaN   
32        Discount Store   Distribution Center               Dog Run   
45                   NaN                   NaN                   NaN   
52                   NaN                   NaN                   NaN   
64                   NaN                   NaN                   NaN   
85    Dim Sum Restaurant                 Diner        Discount Store   
95                   NaN                   NaN                   NaN   

   9th Most Common Venue 10th Most Common Venue  
5                    NaN                    NaN  
11                   NaN                    NaN  
32      Doner Restaurant              Drugstore  
45                   NaN                    NaN  
52                   NaN                    NaN  
64                   NaN                    NaN  
85   Distribution Center          Women's Store  
95                   NaN                    NaN

In [38]:
# CLUSTER #2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
0         North York               1     Food & Drink Shop   
21              York               1                  Park   
35         East York               1                  Park   
61   Central Toronto               1                  Park   
66        North York               1                  Park   
77         Etobicoke               1                  Park   
83   Central Toronto               1                  Park   
91  Downtown Toronto               1                  Park   
98         Etobicoke               1                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                   Park         Women's Store                 Diner   
21                  Pool         Women's Store      Greek Restaurant   
35         Metro Station     Convenience Store         Women's Store   
61              Bus Line           Swim School               Dog Run   
66     Convenience Store         Women's Store            Donut Shop   
77        Sandwich Place      Department Store           Event Space   
83         Women's Store            Donut Shop    Dim Sum Restaurant   
91                 Trail            Playground    Dim Sum Restaurant   
98                 River            Smoke Shop               Dog Run   

   5th Most Common Venue 6th Most Common Venue        7th Most Common Venue  \
0         Discount Store   Distribution Center                      Dog Run   
21          Gourmet Shop  Ethiopian Restaurant            Electronics Store   
35      Doner Restaurant                 Diner               Discount Store   
61    Dim Sum Restaurant                 Diner               Discount Store   
66                 Diner        Discount Store          Distribution Center   
77  Ethiopian Restaurant     Electronics Store  Eastern European Restaurant   
83                 Diner        Discount Store          Distribution Center   
91                 Diner        Discount Store          Distribution Center   
98          Dessert Shop    Dim Sum Restaurant                        Diner   

          8th Most Common Venue 9th Most Common Venue  \
0              Doner Restaurant            Donut Shop   
21  Eastern European Restaurant             Drugstore   
35          Distribution Center               Dog Run   
61          Distribution Center      Doner Restaurant   
66                      Dog Run      Doner Restaurant   
77                    Drugstore            Donut Shop   
83                      Dog Run      Doner Restaurant   
91                      Dog Run      Doner Restaurant   
98               Discount Store   Distribution Center   

         10th Most Common Venue  
0                     Drugstore  
21                   Donut Shop  
35                    Drugstore  
61               Farmers Market  
66  Eastern European Restaurant  
77             Doner Restaurant  
83                    Drugstore  
91                   Donut Shop  
98                Women's Store

In [39]:
# CLUSTER #3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels     1st Most Common Venue  \
1          North York               2     Portuguese Restaurant   
2    Downtown Toronto               2               Coffee Shop   
3          North York               2            Clothing Store   
4    Downtown Toronto               2               Coffee Shop   
7          North York               2                       Gym   
8           East York               2               Pizza Place   
9    Downtown Toronto               2            Clothing Store   
10         North York               2                      Park   
13         North York               2                       Gym   
14          East York               2                      Park   
15   Downtown Toronto               2               Coffee Shop   
16               York               2                     Field   
17          Etobicoke               2                 Pet Store   
18        Scarborough               2            Medical Center   
19       East Toronto               2                     Trail   
20   Downtown Toronto               2               Coffee Shop   
22        Scarborough               2               Coffee Shop   
23          East York               2               Coffee Shop   
24   Downtown Toronto               2               Coffee Shop   
25   Downtown Toronto               2             Grocery Store   
26        Scarborough               2          Hakka Restaurant   
27         North York               2  Mediterranean Restaurant   
28         North York               2               Coffee Shop   
29          East York               2         Indian Restaurant   
30   Downtown Toronto               2               Coffee Shop   
31       West Toronto               2                    Bakery   
33         North York               2            Clothing Store   
34         North York               2    Furniture / Home Store   
36   Downtown Toronto               2               Coffee Shop   
37       West Toronto               2                       Bar   
38        Scarborough               2          Department Store   
39         North York               2                      Café   
40         North York               2             Grocery Store   
41       East Toronto               2          Greek Restaurant   
42   Downtown Toronto               2               Coffee Shop   
43       West Toronto               2                      Café   
44        Scarborough               2                  Bus Line   
46         North York               2             Grocery Store   
47       East Toronto               2            Sandwich Place   
48   Downtown Toronto               2               Coffee Shop   
49         North York               2                      Park   
50         North York               2               Pizza Place   
51        Scarborough               2       American Restaurant   
53         North York               2             Grocery Store   
54       East Toronto               2                      Café   
55         North York               2               Coffee Shop   
56               York               2            Discount Store   
57         North York               2            Baseball Field   
58        Scarborough               2           College Stadium   
59         North York               2          Ramen Restaurant   
60         North York               2             Grocery Store   
62    Central Toronto               2              Home Service   
63               York               2               Pizza Place   
65        Scarborough               2         Indian Restaurant   
67    Central Toronto               2                       Gym   
68    Central Toronto               2                     Trail   
69       West Toronto               2        Mexican Restaurant   
70          Etobicoke               2               Pizza Place   
71        Scarborough               2               Auto Gara

In [40]:
# CLUSTER #4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
12  Scarborough               3                   Bar         Women's Store   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
12            Donut Shop                 Diner        Discount Store   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
12   Distribution Center               Dog Run      Doner Restaurant   

   9th Most Common Venue 10th Most Common Venue  
12             Drugstore                  Field

In [41]:
# CLUSTER #5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
6  Scarborough               4  Fast Food Restaurant          Dessert Shop   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
6        Farmers Market    Falafel Restaurant           Event Space   

  6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
6  Ethiopian Restaurant     Electronics Store  Eastern European Restaurant   

  9th Most Common Venue 10th Most Common Venue  
6             Drugstore             Donut Shop